In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.datasets import mnist

In [2]:
(images, labels), (test_images, test_labels) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [3]:
images = images.reshape((60000, 28*28)).astype("float32")/255
test_images = test_images.reshape((10000, 28*28)).astype("float32")/255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

In [7]:
loss_fn = keras.losses.SparseCategoricalCrossentropy()
loss_tracker = keras.metrics.Mean(name = "mean")
optimizer = keras.optimizers.RMSprop()

class CustomModel(keras.Model):
  def train_step(self, data):
    inputs, targets = data
    with tf.GradientTape() as tape:
      predictions = self(inputs, training = True)
      loss = loss_fn(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))

    loss_tracker.update_state(loss)
    return {"loss": loss_tracker.result()}

  @property
  def metrics(self):
    return [loss_tracker]

In [8]:
inputs = keras.Input(shape = (28*28, ))
features = layers.Dense(512, activation = "relu")(inputs)
features = layers.Dropout(0.5)(features)
outputs = layers.Dense(10, activation = "softmax")(features)
model = CustomModel(inputs, outputs)

model.compile(optimizer = keras.optimizers.RMSprop())
model.fit(train_images, train_labels, epochs = 3)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.4466
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1677
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1270


In [14]:
class myCustomModel(keras.Model):
  def train_step(self, data):
    inputs, targets = data
    with tf.GradientTape() as tape:
      predictions = self(inputs, training = True)
      loss = self.compiled_loss(targets, predictions)
    gradients = tape.gradient(loss, self.trainable_weights)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))

    self.compiled_metrics.update_state(targets, predictions)
    return {m.name: m.result() for m in self.metrics}


In [15]:
inputs = keras.Input(shape = (28 * 28, ))
features = layers.Dense(512, activation = "relu")(inputs)
features = layers.Dropout(0.5)(features)
outputs = layers.Dense(10, activation = "softmax")(features)
model = myCustomModel(inputs, outputs)

model.compile(optimizer = keras.optimizers.RMSprop(),
              loss = keras.losses.SparseCategoricalCrossentropy(),
              metrics = [keras.metrics.SparseCategoricalAccuracy()])
model.fit(train_images, train_labels, epochs = 3)

Epoch 1/3


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:667: UserWarning: `model.compiled_loss()` is deprecated. Instead, use `model.compute_loss(x, y, y_pred, sample_weight, training)`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:642: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - sparse_categorical_accuracy: 0.8654 - loss: 0.1000
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - sparse_categorical_accuracy: 0.9518 - loss: 0.1000
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - sparse_categorical_accuracy: 0.9635 - loss: 0.1000
